In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import json
import time

batch_size = 0
num_classes = 0
epochs = 0
data_augmentation = True
num_predictions = 0
activation = ""
metrics = ""

time = time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))
os.chdir(r'C:\Users\zaclge\PycharmProjects\info7374_ass1')
save_dir = r'C:\Users\zaclge\PycharmProjects\info7374_ass1\saved_models'
model_name = 'keras_cifar10_trained_model.h5'
print(os.getcwd())
def import_json(str_input):
    
    with open(r'C:\Users\zaclge\PycharmProjects\info7374_ass1\configuration','r') as load_json:
        load_dict = json.load(load_json)
        print(load_dict)
        global batch_size, num_classes, epochs, data_augmentation, num_predictions, activation, metrics
        batch_size = load_dict[str_input]["batch_size"]
        num_predictions = load_dict[str_input]["num_predictions"]
        num_classes = load_dict[str_input]["num_classes"]
        epochs = load_dict[str_input]["epochs"]
        data_augmentation = load_dict[str_input]["data_augmentation"]
        activation = load_dict[str_input]["activation"]
        metrics = load_dict[str_input]["metric"]

In [ ]:
def progress(paras):
    # The data, shuffled and split between train and test sets:
    import_json(paras)
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    
    # Convert class vectors to binary class matrices.
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation(activation))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation(activation))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation(activation))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    
    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=[metrics])
    
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    
    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test, y_test),
                  shuffle=True)
    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
    
        # Compute quantities required for feature-wise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)
    
        # Fit the model on the batches generated by datagen.flow().
        model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            workers=4)
    
    # Save model and weights
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name)
    #model.save(model_path)
    print('Saved trained model at %s ' % model_path)
    
    # Score trained model.
    scores = model.evaluate(x_test, y_test, verbose=1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])

In [20]:
progress("one")

{'one': {'batch_size': 128, 'num_classes': 10, 'epochs': 100, 'data_augmentation': True, 'num_predictions': 20, 'activation': 'relu', 'metric': 'accuracy'}, 'two': {'batch_size': 128, 'num_classes': 10, 'epochs': 100, 'data_augmentation': True, 'num_predictions': 20, 'activation': 'relu', 'metric': 'top_k_categorical_accuracy'}, 'three': {'batch_size': 128, 'num_classes': 10, 'epochs': 100, 'data_augmentation': True, 'num_predictions': 20, 'activation': 'sigmoid', 'metric': 'accuracy'}}
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/100
391/391 [==============================] - 127s 325ms/step - loss: 1.9975 - acc: 0.2643 - val_loss: 1.7270 - val_acc: 0.3870
Epoch 2/100
391/391 [==============================] - 126s 321ms/step - loss: 1.7438 - acc: 0.3626 - val_loss: 1.5788 - val_acc: 0.4284
Epoch 3/100
391/391 [==============================] - 126s 322ms/step - loss: 1.6432 - acc: 0.4004 - val_loss: 1.5579 - val_a

391/391 [==============================] - 158s 403ms/step - loss: 0.8063 - acc: 0.7176 - val_loss: 0.7167 - val_acc: 0.7495
Epoch 57/100
391/391 [==============================] - 165s 422ms/step - loss: 0.8078 - acc: 0.7196 - val_loss: 0.7076 - val_acc: 0.7542
Epoch 58/100
391/391 [==============================] - 150s 385ms/step - loss: 0.7972 - acc: 0.7215 - val_loss: 0.6916 - val_acc: 0.7590
Epoch 59/100
391/391 [==============================] - 154s 395ms/step - loss: 0.7927 - acc: 0.7229 - val_loss: 0.7116 - val_acc: 0.7589
Epoch 60/100
391/391 [==============================] - 171s 437ms/step - loss: 0.7836 - acc: 0.7254 - val_loss: 0.6638 - val_acc: 0.7723
Epoch 61/100
391/391 [==============================] - 165s 423ms/step - loss: 0.7834 - acc: 0.7266 - val_loss: 0.7020 - val_acc: 0.7572
Epoch 62/100
391/391 [==============================] - 168s 430ms/step - loss: 0.7861 - acc: 0.7287 - val_loss: 0.7004 - val_acc: 0.7582
Epoch 63/100
391/391 [=========================

In [ ]:
progress("two")